In [2]:
import numpy as np
import tensorflow as tf

In [3]:
tf.__version__

'2.1.0'

In [44]:
EMBED_SIZE = 4
HALF = EMBED_SIZE // 2

encoder_bilstm = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(EMBED_SIZE, return_sequences=True)
)

decoder_lstm = tf.keras.layers.LSTMCell(EMBED_SIZE)

In [5]:
data = np.random.rand(32, 10, 17).astype(np.float32)  # Batch 32, Seqlen 10, Embedding size 17
states = np.random.rand(32, EMBED_SIZE).astype(np.float32)

In [36]:
encoded_output = encoder_bilstm(data)

In [ ]:
encoded_output.shape

In [10]:
output.shape

TensorShape([32, 10, 8])

In [37]:
_, (decoded_hidden, decoder_carry) = decoder_lstm(inputs=tf.convert_to_tensor(data[:, 0, :]), states=[tf.convert_to_tensor(states)]*2)

In [34]:
len(output[1][1])

32

In [17]:
tf.convert_to_tensor(data[:, 0, :]).shape._dims

[Dimension(32), Dimension(17)]

In [38]:
data[:, 0, :].shape

(32, 17)

In [26]:
output[1].shape

TensorShape([10, 4])

TensorShape([32, 10, 4])

In [110]:
def sample(encoded_out, t_carry, v_att, W_h, W_s, b_att):
    """
    encoded_out : Output embeddings from the encoder, over all timesteps.
    t_carry     : Carry/state vector for LSTM at time t.
    """
    
    # Compute attention scores.
    scores = tf.linalg.matmul(
        tf.linalg.matmul(encoded_out, W_h) + 
        tf.expand_dims(tf.linalg.matmul(t_carry, W_s), 1) + 
        b_att,
        tf.expand_dims(v_att, 1)
    )
    attention = tf.nn.softmax(scores, axis=-1)
    
    # Compute context vector at time t.
    context_t = tf.reduce_sum(tf.math.multiply(encoded_out, attention), axis=1)
    
    # Concat, and return for linear layer application ...
    combined = tf.concat([context_t, t_carry], axis=1)
    
    return combined

In [109]:
np.concatenate([np.eye(3), np.eye(3)], axis=1)

array([[1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1.]])

In [104]:
W_h = tf.random.normal([2*EMBED_SIZE, HALF])
W_s = tf.random.normal([EMBED_SIZE, HALF])
b_att = tf.random.normal([HALF])
v_att = tf.random.normal([HALF])

In [119]:
guniform_init = tf.keras.initializers.glorot_uniform()

In [122]:
tf.Variable(guniform_init(shape=(3, 1)))

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-0.4801829 ],
       [-0.21767259],
       [ 1.0301329 ]], dtype=float32)>

In [107]:
decoder_carry.shape

TensorShape([32, 4])

In [89]:
b_att.shape

TensorShape([2])

In [90]:
tf.linalg.matmul(encoded_output, W_h).shape, tf.expand_dims(tf.linalg.matmul(decoder_carry, W_s), 1).shape

(TensorShape([32, 10, 2]), TensorShape([32, 1, 2]))

In [93]:
tf.expand_dims(v_att, axis=1)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[ 0.27558175],
       [-0.9221801 ]], dtype=float32)>

In [114]:
combined = sample(encoded_output, decoder_carry, v_att, W_h, W_s, b_att)

In [112]:
linear_first = tf.keras.layers.Dense(13)
linear_second = tf.keras.layers.Dense(17)

In [115]:
linear_second(linear_first(combined))

<tf.Tensor: shape=(32, 17), dtype=float32, numpy=
array([[ 1.86951190e-01, -1.20361245e+00, -1.72669339e+00,
         3.77934504e+00,  2.27894172e-01,  1.78456807e+00,
         2.65528870e+00,  8.73264253e-01,  3.24787945e-01,
        -5.61868735e-02, -2.30357313e+00, -1.40979970e+00,
         2.53263855e+00,  5.73551893e-01, -2.67674178e-01,
        -1.74926293e+00,  1.91757858e-01],
       [-4.77643192e-01, -1.20872080e+00, -1.17112792e+00,
         2.81317592e+00,  2.78202072e-02,  1.76180863e+00,
         2.45055079e+00,  7.84807503e-01, -4.21361268e-01,
        -2.41879761e-01, -2.25457096e+00, -1.21026337e+00,
         2.02210021e+00,  9.78618264e-01,  2.30674937e-01,
        -2.28214931e+00, -4.93658990e-01],
       [ 4.65392545e-02, -1.09420598e+00, -1.03124309e+00,
         3.59437776e+00, -5.68216853e-02,  1.53625238e+00,
         2.31033182e+00,  9.86307085e-01, -5.64780831e-01,
        -4.66508061e-01, -1.86184347e+00, -5.39180279e-01,
         1.92134011e+00,  8.65786731e-